### Contents
Teaching a Model AND and OR<br>
&emsp;Create Boolean Data<br>
&emsp;Design a Model<br>
&emsp;Train the Model<br>
&emsp;Test the Model<br>
Teaching a Model XOR<br>
&emsp;Create Boolean Data<br>
&emsp;Design a Model<br>
&emsp;Train the Model<br>
&emsp;Test the Model<br>

In [1]:
import random as rnd
from typing import List, Tuple

import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor

import utilities as util


Create Boolean Data

In [8]:
class BooleanDataset(torch.utils.data.Dataset):
    '''
    This class will emulate the function: y = m * x + b
    In machine learning terms m becomes our weight and b becomes the bias.
    '''

    def __init__(self, size: int=1000):
        X = []
        y = []
        count = int(size/2)
        for _ in range(count):
            feature_1 = rnd.randint(0, 1)
            feature_2 = rnd.randint(0, 1)
            label_and = feature_1 and feature_2
            label_or = feature_1 or feature_2
            # Append the and data.
            sample = [feature_1, feature_2]
            X.append(sample)
            y.append(label_and)
            # Append the or data.
            X.append(sample)
            y.append(label_or)

        self.X = np.array(X)   #torch.from_numpy(X)
        self.y = np.array(y)   #torch.from_numpy(y)
        #X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, shuffle=True, stratify=y, random_state=42)

    def __getitem__(self, index):
        return self.X[index, None], self.y[index, None]

    def __len__(self):
        return len(self.X)


#X, y = create_and_or_data(10)
data = BooleanDataset(10)

print('X Dimenstions:', data.X.ndim)
print('X Shape:', data.X.shape)
print(data.X[:5])

print('y Dimenstions:', data.y.ndim)
print('y Shape:', data.y.shape)
print(data.y[:5])

X Dimenstions: 2
X Shape: (10, 2)
[[1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]]
y Dimenstions: 1
y Shape: (10,)
[1 1 0 1 0]


Design a Model

In [ ]:
class AndOrModel(nn.Module):

    def __init__(self):
        super(AndOrModel, self).__init__()

        self.linear1 = nn.Linear(2, 1, bias=True)

    def forward(self, input):
        l1 = self.linear1(input)
        return l1

Create the Model and Print the Initial Parameters

In [ ]:
model = AndOrModel()
util.print_parameters(model)

Train the Model and Print the Results

In [ ]:
config = {
    'epochs': 100,
    'lr': 0.01,
    'loss_function': nn.MSELoss()
}

model, losses = util.train_model(model, config, X, y)

# The loss should decrease with every iteration (epoch) over the training data.
util.print_results(model, losses)

In [ ]:
predictions = [model(torch.from_numpy(X)).detach().item() for X in X_train]
util.plot_data(np.array(X_train), np.array(y_train), np.array(predictions))

### A Quadratic Example

In [ ]:
def create_quadratic_data(a:float, b: float, c:float) -> Tuple[List[float], List[float]]:
    X = [float(x) for x in range(-10, 11)]
    y = [a*(x**2)+(b*x)+c for x in X]
    X, y = np.array(X, dtype=np.float32), np.array(y, np.float32)
    X = np.reshape(X, (len(X), 1))
    y = np.reshape(y, (len(y), 1))
    return X, y

X_train, y_train = create_quadratic_data(5, 2, 3)

print('X_train Dimenstions:',X_train.ndim)
print('X_train Shape:', X_train.shape)
print(X_train[:2])

print('y_train Dimenstions:',y_train.ndim)
print('y_train Shape:', y_train.shape)
print(y_train[:2])

In [ ]:
plot_data(X_train, y_train)

In [ ]:
model = LinearRegressionModel()
model, losses = train_model(model, X_train, y_train)

# The loss should decrease with every iteration (epoch) over the training data.
print_results(model, losses)

In [ ]:
predictions = [model(torch.from_numpy(X)).detach().item() for X in X_train]
plot_data(np.array(X_train), np.array(y_train), np.array(predictions))

In [ ]:
class QuadraticRegressionModel(nn.Module):

    def __init__(self):
        super(QuadraticRegressionModel, self).__init__()

        self.linear1 = nn.Linear(1, 6, bias=True)
        self.linear2 = nn.Linear(6, 6, bias=True)
        self.linear3 = nn.Linear(6, 1, bias=True)

    def forward(self, x, log=False):
        x = F.dropout(F.relu(self.linear1(x)), p=0.5)
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        #out = self.linear1(input)
        #out = F.relu(out)
        #out = F.dropout(out, p=0.5)
        #out = self.linear2(out)
        #out = F.relu(out)
        #out = self.linear3(out)
        return x

Test the Untrained Model with a Single Prediction

In [ ]:
model = QuadraticRegressionModel()
X = torch.tensor([1], dtype=torch.float32)
prediction = model(X, log=True)
print(X)
print(prediction)

In [ ]:
model = QuadraticRegressionModel()
model, losses = train_model(model, X_train, y_train)

# The loss should decrease with every iteration (epoch) over the training data.
print_results(model, losses)

Plot labels and predictions

In [ ]:
#model.eval() # Tell the model we are evaluating it so that it does not learn or dropout.
predictions = [model(torch.from_numpy(X)).detach().item() for X in X_train]

plot_data(np.array(X_train), np.array(y_train), np.array(predictions))